<a href="https://colab.research.google.com/github/woo2201/sampleCases/blob/main/Attribution_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [47]:
cd /content/gdrive/MyDrive/dataset/Lifeplanet/Analytics

/content/gdrive/MyDrive/dataset/Lifeplanet/Analytics


In [48]:
import os
import glob
import numpy as np
import pandas as pd

In [49]:
path = "/content/gdrive/MyDrive/dataset/Lifeplanet/Analytics/"
filenames = glob.glob(path + "*.csv")
data = [var.split('.')[0].split('/')[-1] for var in filenames]

# dynamically allocate df names.
for name in zip(filenames, data):
        loc = list(name)[0]
        val = list(name)[1]
        globals()[val] = pd.read_csv(loc, sep='delimiter', header=None, engine = 'python')

In [50]:
data

['Analytics_3',
 'Analytics_2',
 'Analytics_4',
 'Analytics_1',
 'Analytics_5',
 'hee']

In [51]:
def preprocessing(df):

    df = df[6:]
    df.columns = df.columns.astype(str)
    df.rename(columns = {'0':'vector_1'}, inplace = True)

    return df

In [52]:
df = pd.concat([preprocessing(Analytics_1),
                preprocessing(Analytics_2),
                preprocessing(Analytics_3),
                preprocessing(Analytics_4),
                preprocessing(Analytics_5)])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [53]:
df = df.reset_index()

In [54]:
df = df.drop(['index'], axis = 1)

In [55]:
df = df['vector_1'].str.split(',', expand = True)

In [56]:
df.columns = df.columns.astype(str)

In [57]:
df = df.add_prefix('temp_')

In [58]:
df.fillna('-', inplace = True)

In [59]:
df['rslt'] = df['temp_1'] + df['temp_2'] + df['temp_3'] + df['temp_4'] + df['temp_5']

In [60]:
df = pd.concat([df, df['rslt'].str.split('₩', expand = True).add_prefix('temps_')], axis = 1)

In [61]:
df = df.drop(['temp_1', 'temp_2', 'temp_3', 'temp_4', 'temp_5', 'rslt'], axis = 1)

In [62]:
import re

def clean_text(inputString):

  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inputString)
  text_rmv = ' '.join(text_rmv.split())

  return text_rmv

In [63]:
df['temps_0'] = df['temps_0'].apply(lambda x: clean_text(x))
df['temps_1'] = df['temps_1'].apply(lambda x: clean_text(x))

In [64]:
df['temp_seq'] = df['temp_0'].str.split('>', expand = False)

In [65]:
df['temp_seq'] = df['temp_seq'].apply(lambda x : [i.strip() for i in x])

In [66]:
# df['temp_seq'][1335:5151]

In [67]:
from itertools import groupby
df['temp_seq'] = df['temp_seq'].apply(lambda x : [k for k, _ in groupby(x)])

In [68]:
df = df[['temp_seq', 'temps_0', 'temps_1']]

In [69]:
df

,temp_seq,temps_0,temps_1
0,[(direct)],28627,55391433
1,"[kakaopay, (direct)]",13174,90300
2,"[e-insmarket.or.kr, (direct)]",13013,1964547
3,[(direct)],12960,29049441
4,"[pincrux, (direct)]",10948,22311620
...,...,...,...
24995,"[google, e-insmarket.or.kr, (direct), criteo, ...",1,0
24996,"[google, e-insmarket.or.kr, (direct), criteo, ...",1,0
24997,"[google, e-insmarket.or.kr, (direct), criteo, ...",1,0
24998,"[google, e-insmarket.or.kr, (direct), criteo, ...",1,0


In [70]:
cols = pd.DataFrame([pd.Series(x) for x in df.temp_seq])
cols.columns = ['chnn_{}'.format(x+1) for x in cols.columns]

In [71]:
df = pd.concat([cols, df[['temps_0','temps_1']] ], axis = 1).fillna('-')

In [72]:
df

,chnn_1,chnn_2,chnn_3,chnn_4,chnn_5,chnn_6,chnn_7,chnn_8,chnn_9,chnn_10,...,chnn_60,chnn_61,chnn_62,chnn_63,chnn_64,chnn_65,chnn_66,chnn_67,temps_0,temps_1
0,(direct),-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,28627,55391433
1,kakaopay,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,13174,90300
2,e-insmarket.or.kr,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,13013,1964547
3,(direct),-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,12960,29049441
4,pincrux,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,10948,22311620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,google,e-insmarket.or.kr,(direct),criteo,(direct),google,(direct),-,-,-,...,-,-,-,-,-,-,-,-,1,0
24996,google,e-insmarket.or.kr,(direct),criteo,(direct),google,(direct),e-insmarket.or.kr,-,-,...,-,-,-,-,-,-,-,-,1,0
24997,google,e-insmarket.or.kr,(direct),criteo,(direct),google,(direct),e-insmarket.or.kr,(direct),-,...,-,-,-,-,-,-,-,-,1,0
24998,google,e-insmarket.or.kr,(direct),criteo,(direct),google,(direct),e-insmarket.or.kr,(direct),-,...,-,-,-,-,-,-,-,-,1,0


In [73]:
df['chnn_1'].value_counts # 242 unique_values

<bound method IndexOpsMixin.value_counts of 0                 (direct)
1                 kakaopay
2        e-insmarket.or.kr
3                 (direct)
4                  pincrux
               ...        
24995               google
24996               google
24997               google
24998               google
24999               google
Name: chnn_1, Length: 25000, dtype: object>

In [74]:
df['acc_prc'] = round( df['temps_0'].astype(int) / df['temps_0'].astype(int).sum(), 5)

In [75]:
df = df[df.chnn_2 != '-']

In [76]:
df

,chnn_1,chnn_2,chnn_3,chnn_4,chnn_5,chnn_6,chnn_7,chnn_8,chnn_9,chnn_10,...,chnn_61,chnn_62,chnn_63,chnn_64,chnn_65,chnn_66,chnn_67,temps_0,temps_1,acc_prc
1,kakaopay,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,13174,90300,0.04195
2,e-insmarket.or.kr,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,13013,1964547,0.04143
4,pincrux,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,10948,22311620,0.03486
6,e-insmarket.or.kr,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,6835,4944680,0.02176
9,kakaopay,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,5069,24200,0.01614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,google,e-insmarket.or.kr,(direct),criteo,(direct),google,(direct),-,-,-,...,-,-,-,-,-,-,-,1,0,0.00000
24996,google,e-insmarket.or.kr,(direct),criteo,(direct),google,(direct),e-insmarket.or.kr,-,-,...,-,-,-,-,-,-,-,1,0,0.00000
24997,google,e-insmarket.or.kr,(direct),criteo,(direct),google,(direct),e-insmarket.or.kr,(direct),-,...,-,-,-,-,-,-,-,1,0,0.00000
24998,google,e-insmarket.or.kr,(direct),criteo,(direct),google,(direct),e-insmarket.or.kr,(direct),-,...,-,-,-,-,-,-,-,1,0,0.00000
